In [1]:
!python -m pip install langgraph langchain==1.1.0 openai pytest python-dotenv

  Using cached python_dotenv-1.2.1-py3-none-any.whl.metadata (25 kB)
  Using cached pluggy-1.6.0-py3-none-any.whl.metadata (4.8 kB)
Using cached python_dotenv-1.2.1-py3-none-any.whl (21 kB)

   ---------- ----------------------------- 1/4 [pluggy]
   ------------------------------ --------- 3/4 [pytest]
   ------------------------------ --------- 3/4 [pytest]
   ------------------------------ --------- 3/4 [pytest]
   ------------------------------ --------- 3/4 [pytest]
   ------------------------------ --------- 3/4 [pytest]
   ------------------------------ --------- 3/4 [pytest]
   ---------------------------------------- 4/4 [pytest]



  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 25.3 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import subprocess
from typing import TypedDict, Optional
from langgraph.graph import StateGraph, END
from openai import OpenAI
from dotenv import load_dotenv
import os

load_dotenv()

# -------------------------
# Configuração do modelo
# -------------------------

base_url = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com/20231130/actions/v1"  # ou sua URL
api_key = os.getenv("OCI_API_KEY")

client = OpenAI(
    base_url=base_url,
    api_key=api_key
)

MODEL = "xai.grok-4-fast-non-reasoning"  # ou modelo compatível


c:\Users\Amanda Machado\AppData\Local\Programs\Python\Python314\Lib\site-packages\langchain_core\_api\deprecation.py:25: UserWarning: Core Pydantic V1 functionality isn't compatible with Python 3.14 or greater.
  from pydantic.v1.fields import FieldInfo as FieldInfoV1


In [2]:
# -------------------------
# Estado do agente
# -------------------------

class AgentState(TypedDict):
    task: str
    code: Optional[str]
    error: Optional[str]
    attempts: int

MAX_ATTEMPTS = 3

In [3]:
# -------------------------
# Node 1 - Gerar Código
# -------------------------

def generate_code(state: AgentState):
    prompt = f"""
    Gere código Python para a tarefa abaixo.
    Retorne apenas código Python puro.

    Tarefa:
    {state['task']}
    """

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}]
    )

    code = response.choices[0].message.content

    return {
        "code": code,
        "attempts": state["attempts"] + 1
    }

In [18]:
import sys
import subprocess
# -------------------------
# Node 2 - Executar Testes
# -------------------------
def clean_code(code):
    return code.replace("```python", "").replace("```", "").strip()

def execute_code(state: AgentState):
    cleaned = clean_code(state["code"])
    with open("./solution.py", "w") as f:
        f.write(cleaned)

    result = subprocess.run(
        [sys.executable, "-m", "pytest", "test_solution.py"],
        capture_output=True,
        text=True
    )

    if result.returncode == 0:
        return {"error": None}
    else:
        return {"error": result.stdout + result.stderr}


In [5]:
# -------------------------
# Node 3 - Reflexão
# -------------------------

def reflect(state: AgentState):
    prompt = f"""
    A tarefa era:
    {state['task']}

    O código gerado foi:
    {state['code']}

    O erro retornado foi:
    {state['error']}

    Analise o erro e gere uma versão corrigida.
    Retorne apenas código Python.
    """

    response = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": prompt}]
    )

    new_code = response.choices[0].message.content

    return {
        "code": new_code,
        "attempts": state["attempts"] + 1
    }


In [6]:
# -------------------------
# Lógica de decisão
# -------------------------

def should_continue(state: AgentState):
    if state["error"] is None:
        return END
    if state["attempts"] >= MAX_ATTEMPTS:
        return END
    return "reflect"

In [19]:
# -------------------------
# Construção do Grafo
# -------------------------

graph = StateGraph(AgentState)

graph.add_node("generate", generate_code)
graph.add_node("execute", execute_code)
graph.add_node("reflect", reflect)

graph.set_entry_point("generate")

graph.add_edge("generate", "execute")
graph.add_conditional_edges(
    "execute",
    should_continue,
    {
        "reflect": "reflect",
        END: END
    }
)

graph.add_edge("reflect", "execute")

app = graph.compile()

In [20]:
# -------------------------
# Execução
# -------------------------


initial_state = {
    "task": "Implemente uma função soma(a, b) que retorna a soma.",
    "code": None,
    "error": None,
    "attempts": 0
}

result = app.invoke(initial_state)
print("Final State:", result)

Final State: {'task': 'Implemente uma função soma(a, b) que retorna a soma.', 'code': '```python\ndef soma(a, b):\n    return a + b\n```', 'error': '\x1b============================= test session starts =============================\x1b\nplatform win32 -- Python 3.14.2, pytest-9.0.2, pluggy-1.6.0\nrootdir: c:\\Users\\Amanda Machado\\Documents\\Mine\\PÃ³s - Impacta\\1- cÃ³digos\\SessÃ£o 1\nplugins: anyio-4.12.1, langsmith-0.7.6\ncollected 0 items\n\n\x1b============================ \x1bno tests ran\x1b\x1b in 0.01s\x1b\x1b ============================\x1b\n\x1bERROR: file or directory not found: test_solution.py\n\x1b\n', 'attempts': 3}


In [21]:
print("Tarefa:")
print(result["task"])

print("\nCódigo Gerado:")
print(result["code"])

print("\nErro:")
print(result["error"])

print("\nTentativas:")
print(result["attempts"])

Tarefa:
Implemente uma função soma(a, b) que retorna a soma.

Código Gerado:
```python
def soma(a, b):
    return a + b
```

Erro:
============================= test session starts =============================
platform win32 -- Python 3.14.2, pytest-9.0.2, pluggy-1.6.0
rootdir: c:\Users\Amanda Machado\Documents\Mine\PÃ³s - Impacta\1- cÃ³digos\SessÃ£o 1
plugins: anyio-4.12.1, langsmith-0.7.6
collected 0 items

============================ no tests ran in 0.01s ============================
ERROR: file or directory not found: test_solution.py



Tentativas:
3
